# Annotate Images and map to FER emotion labels

- Author: Ankita Jaswal

In [ ]:
# prompt: mount drive

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


### Below are the steps to create an emotion annotator:

1. Face Extraction:

The first part of this code extracts faces from a set of images and saves them in a directory. This step is using a pre-trained deep learning model to detect faces in the images.

2. Emotion Detection:

3. Run the script, and it will first extract faces from the images using the face detection model and then detect emotions in the extracted faces using the FER library.

  The script will create subdirectories for each predicted emotion and save the faces in their respective directories.

  The script will also print the predicted emotions for each extracted face.

In [ ]:
!pip install  opencv-python==4.7.0.72


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.8/61.8 MB 13.5 MB/s eta 0:00:00
  Attempting uninstall: opencv-python
    Found existing installation: opencv-python 4.8.0.76
    Uninstalling opencv-python-4.8.0.76:
      Successfully uninstalled opencv-python-4.8.0.76


In [ ]:
!pip install fer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 15.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for ffmpeg: filename=ffmpeg-1.4-py3-none-any.whl size=6082 sha256=6266ce5870dcd4ad55d6297de74d7845f7d1d4da2609261c86bdb795b07a6650
  Stored in directory: /root/.cache/pip/wheels/8e/7a/69/cd6aeb83b126a7f04cbe7c9d929028dc52a6e7d525ff56003a
Successfully built ffmpeg


In [ ]:
# Import libraries
import os
import cv2
import numpy as np
from fer import FER
import matplotlib.pyplot as plt

# Define paths
images_directory = "/content/drive/MyDrive/CMPE 258 - Deep Learning/fer/emotions"

# Create directory 'emotions' if it does not exist
if not os.path.exists(images_directory):
    print("New directory created")
    os.makedirs(images_directory)

# Read the model
model = cv2.dnn.readNetFromCaffe("/content/drive/MyDrive/CMPE 258 - Deep Learning/model_data/deploy.prototxt", "/content/drive/MyDrive/CMPE 258 - Deep Learning/model_data/weights.caffemodel")

# Create directory 'emo_faces' if it does not exist
if not os.path.exists("/content/drive/MyDrive/CMPE 258 - Deep Learning/emotion_faces"):
    print("New directory created")
    os.makedirs("/content/drive/MyDrive/CMPE 258 - Deep Learning/emotion_faces")

# Initialize the FER detector
emo_detector = FER()

# List to store detected faces
detected_faces = []

# Loop through all images and strip out faces
for file in os.listdir("/content/drive/MyDrive/CMPE 258 - Deep Learning/emotic/emodb_small/images"):
    file_name, file_extension = os.path.splitext(file)
    if (file_extension in ['.png', '.jpg']):
        image = cv2.imread("/content/drive/MyDrive/CMPE 258 - Deep Learning/emotic/emodb_small/images/" + file)

        # Check if image reading was successful
        if image is not None:
            (h, w) = image.shape[:2]
            blob = cv2.dnn.blobFromImage(cv2.resize(image, (300, 300)), 1.0, (300, 300), (104.0, 177.0, 123.0))

            model.setInput(blob)
            detections = model.forward()

            # List to store detected faces in the current image
            detected_faces_in_image = []

            # Identify each face
            for i in range(0, detections.shape[2]):
                box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
                (startX, startY, endX, endY) = box.astype("int")

                confidence = detections[0, 0, i, 2]

                # If confidence > 0.5, save it as a separate file
                if (confidence > 0.5):
                    frame = image[startY:endY, startX:endX]
                    if frame.size != 0:  # Check if the frame is not empty
                        # Generate a unique filename for each face
                        face_filename = str(i) + '_' + file
                        cv2.imwrite("/content/drive/MyDrive/CMPE 258 - Deep Learning/emotion_faces/" + face_filename, frame)
                        print("Image " + file + " saved successfully as", face_filename)
                        detected_faces_in_image.append(face_filename)

            if detected_faces_in_image:
                detected_faces.extend(detected_faces_in_image)

            # If no faces were detected, print a message and skip the image
            if not detected_faces_in_image:
                print("No faces found in " + file + ". Skipping...")
        else:
            print("Error reading " + file + ". Skipping...")

print("Faces extracted from all images")

# Check if faces were detected before running emotion detection
if detected_faces:
    # Process the detected faces for emotion detection
    for face_filename in detected_faces:
        test_image = plt.imread("/content/drive/MyDrive/CMPE 258 - Deep Learning/emotion_faces/" + face_filename)

        if test_image is None:
            print("Error reading " + face_filename + ". Skipping...")
            continue
        # Capture all the emotions in the image
        captured_emotions = emo_detector.detect_emotions(test_image)

        # Get the dominant emotion
        dominant_emotion, emotion_score = emo_detector.top_emotion(test_image)

        # Check if the dominant emotion is None
        if dominant_emotion is not None:
            # Create a directory for the predicted emotion if it doesn't exist
            output_directory = os.path.join(images_directory, dominant_emotion)
            os.makedirs(output_directory, exist_ok=True)

            # Save the image to the appropriate directory
            output_path = os.path.join(output_directory, face_filename)
            plt.imsave(output_path, test_image)

            # Print the predicted emotion for each image
            print(f"Image: {face_filename}, Predicted Emotion: {dominant_emotion}, Emotion Score: {emotion_score}")
else:
    print("No faces detected in any of the images.")


Image 1qi7lh7wbp8f7owcp3.jpg saved successfully as 0_1qi7lh7wbp8f7owcp3.jpg
Image 1qi7lh7wbp8f7owcp3.jpg saved successfully as 1_1qi7lh7wbp8f7owcp3.jpg
Image 3yov006j3mbflkl8rk.jpg saved successfully as 0_3yov006j3mbflkl8rk.jpg
Image era4vdcufb70khk28u.jpg saved successfully as 0_era4vdcufb70khk28u.jpg
Image era4vdcufb70khk28u.jpg saved successfully as 1_era4vdcufb70khk28u.jpg
Image 1t6pchzx451rnrurp0.jpg saved successfully as 0_1t6pchzx451rnrurp0.jpg
No faces found in a3ujgmu078f13faqfe.jpg. Skipping...
Image 47tlzgd102rlt5evoo.jpg saved successfully as 0_47tlzgd102rlt5evoo.jpg
No faces found in 64ruxt2v30sis3bp0q.jpg. Skipping...
No faces found in 8ze4iuoys263mhil2v.jpg. Skipping...
No faces found in 7n2a02k8lqoh12azp0.jpg. Skipping...
Image bwv9pojc7kbqavlh0a.jpg saved successfully as 0_bwv9pojc7kbqavlh0a.jpg
Image b7zgc6cw8ma9xmhinn.jpg saved successfully as 0_b7zgc6cw8ma9xmhinn.jpg
Image b7zgc6cw8ma9xmhinn.jpg saved successfully as 1_b7zgc6cw8ma9xmhinn.jpg
Image b6uw3wotb2mhqtxwf4